# 创建用户身份信息表并上传示例数据

本笔记本用于：
1. 创建 DynamoDB 表来存储用户身份信息（风险偏好、投资年限等）
2. 上传示例的用户身份数据
3. 验证数据是否正确上传

## 导入必要的库

In [1]:
import boto3
import yaml
import os
import json
from datetime import datetime
import pandas as pd
from decimal import Decimal
import time

## 设置 AWS 连接

In [2]:
# 创建 DynamoDB 客户端和资源
session = boto3.Session()
region = session.region_name
dynamodb_client = boto3.client("dynamodb", region_name=region)
dynamodb_resource = boto3.resource("dynamodb", region_name=region)
smm_client = boto3.client("ssm")

print(f"AWS Region: {region}")

AWS Region: us-east-1


## 定义知识库名称和表名

In [3]:
kb_name = "fsi-fund-knowledge"
table_name = "user_profile"
pk_item = "user_id"

## 创建 DynamoDB 表

In [4]:
def create_dynamodb_table(kb_name, table_name, pk_item):
    """创建 DynamoDB 表（只有主键，没有排序键）
    Args:
        kb_name: 知识库名称，用于创建 SSM 参数
        table_name: 表名
        pk_item: 主键名称
    """
    try:
        table = dynamodb_resource.create_table(
            TableName=table_name,
            KeySchema=[
                {"AttributeName": pk_item, "KeyType": "HASH"},
            ],
            AttributeDefinitions=[
                {"AttributeName": pk_item, "AttributeType": "S"},
            ],
            BillingMode="PAY_PER_REQUEST",  # 使用按需容量模式
        )

        # 等待表创建完成
        print(f"正在创建表 {table_name}...")
        table.wait_until_exists()
        print(f"表 {table_name} 创建成功！")
        
        # 在参数存储中保存表名
        smm_client.put_parameter(
            Name=f"{kb_name}-{table_name}-table-name",
            Description=f"{kb_name} {table_name} 表名",
            Value=table_name,
            Type="String",
            Overwrite=True,
        )
        print(f"参数 {kb_name}-{table_name}-table-name 创建成功！")
        return table
    except dynamodb_client.exceptions.ResourceInUseException:
        print(f"表 {table_name} 已存在，跳过表创建步骤")
        # 在参数存储中保存表名
        smm_client.put_parameter(
            Name=f"{kb_name}-{table_name}-table-name",
            Description=f"{kb_name} {table_name} 表名",
            Value=table_name,
            Type="String",
            Overwrite=True,
        )
        print(f"参数 {kb_name}-{table_name}-table-name 创建成功！")
        return dynamodb_resource.Table(table_name)

In [5]:
# 创建用户身份信息表
user_profile_table = create_dynamodb_table(kb_name, table_name, pk_item)

正在创建表 user_profile...
表 user_profile 创建成功！
参数 fsi-fund-knowledge-user_profile-table-name 创建成功！


## 准备示例用户身份信息数据

In [6]:
# 创建示例用户身份信息数据
sample_profiles = [
    {
        "user_id": "user001",
        "name": "张三",
        "age": 35,
        "risk_preference": "中等风险",  # 低风险、中等风险、高风险
        "investment_horizon": 5,  # 投资年限（年）
        "investment_goal": "子女教育",  # 投资目标
        "annual_income": Decimal("300000.00"),  # 年收入
        "total_assets": Decimal("1500000.00"),  # 总资产
        "last_updated": datetime.now().isoformat()
    },
    {
        "user_id": "user002",
        "name": "李四",
        "age": 45,
        "risk_preference": "低风险",
        "investment_horizon": 10,
        "investment_goal": "退休规划",
        "annual_income": Decimal("500000.00"),
        "total_assets": Decimal("3000000.00"),
        "last_updated": datetime.now().isoformat()
    },
    {
        "user_id": "user003",
        "name": "王五",
        "age": 28,
        "risk_preference": "高风险",
        "investment_horizon": 3,
        "investment_goal": "财富增值",
        "annual_income": Decimal("200000.00"),
        "total_assets": Decimal("800000.00"),
        "last_updated": datetime.now().isoformat()
    }
]

## 上传示例数据到 DynamoDB 表

In [7]:
def upload_items_to_dynamodb(table, items):
    """上传多个项目到 DynamoDB 表
    Args:
        table: DynamoDB 表资源
        items: 要上传的项目列表
    """
    for item in items:
        try:
            table.put_item(Item=item)
            print(f"成功上传项目: {item['user_id']} - {item['name']}")
        except Exception as e:
            print(f"上传项目时出错: {str(e)}")
            print(f"项目: {item}")

In [8]:
# 上传示例数据
upload_items_to_dynamodb(user_profile_table, sample_profiles)

成功上传项目: user001 - 张三
成功上传项目: user002 - 李四
成功上传项目: user003 - 王五


## 验证数据是否正确上传

In [9]:
def get_user_profile(table, user_id):
    """获取用户身份信息
    Args:
        table: DynamoDB 表资源
        user_id: 用户ID
    Returns:
        用户身份信息
    """
    response = table.get_item(
        Key={
            "user_id": user_id
        }
    )
    return response.get("Item")

In [10]:
# 查询用户001的身份信息
user001_profile = get_user_profile(user_profile_table, "user001")
print("用户001的身份信息:")
print(json.dumps(user001_profile, indent=2, default=str))

用户001的身份信息:
{
  "last_updated": "2025-05-27T17:20:09.912674",
  "user_id": "user001",
  "investment_horizon": "5",
  "annual_income": "300000",
  "name": "\u5f20\u4e09",
  "investment_goal": "\u5b50\u5973\u6559\u80b2",
  "total_assets": "1500000",
  "age": "35",
  "risk_preference": "\u4e2d\u7b49\u98ce\u9669"
}


In [11]:
# 查询用户002的身份信息
user002_profile = get_user_profile(user_profile_table, "user002")
print("用户002的身份信息:")
print(json.dumps(user002_profile, indent=2, default=str))

用户002的身份信息:
{
  "last_updated": "2025-05-27T17:20:09.912684",
  "user_id": "user002",
  "investment_horizon": "10",
  "annual_income": "500000",
  "name": "\u674e\u56db",
  "investment_goal": "\u9000\u4f11\u89c4\u5212",
  "total_assets": "3000000",
  "age": "45",
  "risk_preference": "\u4f4e\u98ce\u9669"
}


In [12]:
# 查询用户003的身份信息
user003_profile = get_user_profile(user_profile_table, "user003")
print("用户003的身份信息:")
print(json.dumps(user003_profile, indent=2, default=str))

用户003的身份信息:
{
  "last_updated": "2025-05-27T17:20:09.912686",
  "user_id": "user003",
  "investment_horizon": "3",
  "annual_income": "200000",
  "name": "\u738b\u4e94",
  "investment_goal": "\u8d22\u5bcc\u589e\u503c",
  "total_assets": "800000",
  "age": "28",
  "risk_preference": "\u9ad8\u98ce\u9669"
}


## 更新 prereqs_config.yaml 文件

请在 `fund-advisor-agent-strands/prereqs/prereqs_config.yaml` 文件中添加以下内容：

```yaml
  # 用户身份信息表
  - table_name: "user_profile"
    pk_item: "user_id"
    sk_item: ""
```